In [ ]:
import time
from random import randint

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [ ]:
# Enter an Instagram profile name as a string below
# And then un-comment the line

# profile_name = ''

In [ ]:
instagram_url = f'https://www.instagram.com/{profile_name}'
max_scroll_secs = 300

In [ ]:
snapshots = []
def snapshot(browser):
    global snapshots
    snapshots.append(browser.page_source)

In [ ]:
# Attribution for scrolling mechanism:
# https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python#27760083

def scroll_and_snapshot(browser, max_scroll_secs):
    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
    
    end_time = time.time() + max_scroll_secs
    
    while time.time() < end_time:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height <= last_height:
            snapshot(browser)
            break
        # Create HTML snapshot
        snapshot(browser)
        last_height = new_height

In [ ]:
def collect_image_tags(snapshots):
    
    image_tags = []
    
    for html in snapshots:

        soup = BeautifulSoup(html, 'html.parser')
        image_tags += soup.find_all('img', class_='FFVAD')
        
    return set(image_tags)

In [ ]:
def capture(snapshots, browser):
        
    image_tags = collect_image_tags(snapshots)

    for index, image in enumerate(image_tags):
        browser.get(image['src'])
        images = browser.find_elements_by_tag_name('img')
        # Edit file name f-string below  as needed
        images[0].screenshot(f'./screenshot_{index}.png')
        wait_time = randint(1, 2)
        time.sleep(wait_time)

In [ ]:
# If you need to download/update chromedriver
# https://sites.google.com/a/chromium.org/chromedriver/

browser = webdriver.Chrome('../utilities/chromedriver')

browser.get(instagram_url)

scroll_and_snapshot(browser, max_scroll_secs)

# ONLY call line below if you want to risk saving a TON of images
# capture(snapshots, browser)

# 2 lines below report # of tags captured w/o saving images
images = collect_image_tags(snapshots)
print(f'Image Links Captured: {len(images)}')

browser.quit()